In [1]:
import mysql.connector
import pandas as pd
import csv

ModuleNotFoundError: No module named 'mysql'

In [2]:
# 数据库信息
host = '39.106.2.176'
port = 3306
user = 'tju'
password = '123456'
database = 'dyqc'

In [3]:
# 连接数据库
conn = mysql.connector.connect(host=host, port=port, user=user, password=password, database=database, charset='utf8')
if conn:
    print("Database connected successfully.")
cur = conn.cursor()

Database connected successfully.


In [4]:
# 从qc_task_detail表获取不同投放团队的计划标识字段ad_id
# 三种情况：(1)部分含TJU计划; (2)全部为TJU计划; (3)不含TJU计划
# (1)部分含TJU计划
def Get_ad_id_part_TJU(advertiser_id, campaign_id_list):
    ad_id_list = list()          # 存储ad_id字段
    for campaign_id in campaign_id_list:
        sql = "SELECT * FROM qc_task_detail WHERE advertiser_id = %s AND campaign_id = %s"
        cur.execute(sql, (advertiser_id, campaign_id))
        row = cur.fetchone()
        while row:
            ad_id_list.append(row[2])
            row = cur.fetchone()
    return ad_id_list

In [5]:
# 从qc_task_data_2111表获取相关数据
# (2)(3)全部为/不含TJU计划
def Get_task_data(advertiser_id):
    time_lb = '2021-11-01 00:00:00'
    time_ub = '2021-11-08 00:00:00'
    df = pd.DataFrame(columns = ['id', 'advertiser_id', 'ad_id', 'time', 'stat_cost','show_cnt', 'ctr', 'cpm_platform', 'click_cnt', 'pay_order_count', 'create_order_amount', 'prepay_and_pay_order_roi', 'prepay_order_count', 'prepay_order_amount', 'create_order_count', 'pay_order_amount', 'create_order_roi', 'dy_follow', 'convert_cnt', 'convert_cost', 'convert_rate', 'dy_share', 'dy_comment', 'dy_like', 'live_pay_order_cost_per_order', 'luban_live_enter_cnt', 'live_watch_one_minute_count', 'live_fans_club_join_cnt', 'luban_live_slidecart_click_cnt', 'luban_live_click_product_cnt', 'luban_live_comment_cnt', 'luban_live_share_cnt', 'luban_live_gift_cnt', 'luban_live_gift_amount', 'create_time', 'update_time'])
    sql = "SELECT * FROM qc_task_data_2111 WHERE advertiser_id = %s AND time >= %s AND time < %s"
    cur.execute(sql, (advertiser_id, time_lb, time_ub))
    row = cur.fetchall()
    if len(row) != 0:
        temp = pd.DataFrame(row, columns = ['id', 'advertiser_id', 'ad_id', 'time', 'stat_cost','show_cnt', 'ctr', 'cpm_platform', 'click_cnt', 'pay_order_count', 'create_order_amount', 'prepay_and_pay_order_roi', 'prepay_order_count', 'prepay_order_amount', 'create_order_count', 'pay_order_amount', 'create_order_roi', 'dy_follow', 'convert_cnt', 'convert_cost', 'convert_rate', 'dy_share', 'dy_comment', 'dy_like', 'live_pay_order_cost_per_order', 'luban_live_enter_cnt', 'live_watch_one_minute_count', 'live_fans_club_join_cnt', 'luban_live_slidecart_click_cnt', 'luban_live_click_product_cnt', 'luban_live_comment_cnt', 'luban_live_share_cnt', 'luban_live_gift_cnt', 'luban_live_gift_amount', 'create_time', 'update_time'])
        df = df.append(temp, ignore_index=True)
        print(df)

In [6]:
# 将qc_task_data_2111表中的数据存储到csv文件中
# (2)(3)全部为/不含TJU计划
def Save_task_data(advertiser_id, advertiser_name):
    fileLoc = "Data/task_data/" + advertiser_name + ".csv"
    f = open(fileLoc, 'w', encoding="utf-8", newline='')
    csv_writer = csv.writer(f)
    csv_writer.writerow(['id', 'advertiser_id', 'ad_id', 'time', 'stat_cost','show_cnt', 'ctr', 'cpm_platform', 'click_cnt', 'pay_order_count', 'create_order_amount', 'prepay_and_pay_order_roi', 'prepay_order_count', 'prepay_order_amount', 'create_order_count', 'pay_order_amount', 'create_order_roi', 'dy_follow', 'convert_cnt', 'convert_cost', 'convert_rate', 'dy_share', 'dy_comment', 'dy_like', 'live_pay_order_cost_per_order', 'luban_live_enter_cnt', 'live_watch_one_minute_count', 'live_fans_club_join_cnt', 'luban_live_slidecart_click_cnt', 'luban_live_click_product_cnt', 'luban_live_comment_cnt', 'luban_live_share_cnt', 'luban_live_gift_cnt', 'luban_live_gift_amount', 'create_time', 'update_time'])
    
    time_lb = '2021-11-01 00:00:00'
    time_ub = '2021-11-08 00:00:00'
    sql = "SELECT * FROM qc_task_data_2111 WHERE advertiser_id = %s AND time >= %s AND time < %s"
    cur.execute(sql, (advertiser_id, time_lb, time_ub))
    row = cur.fetchall()
    if len(row) != 0:
        csv_writer.writerows(row)
#         print(row)
    f.close()
    print("File: " + advertiser_name + " saving complete.")

In [7]:
# (1)部分含TJU计划
# 通过之前获取的ad_id字段，从qc_task_data_2111表获取相关数据
def Get_task_data_part_TJU(ad_id_list):
    count = 1
    length = len(ad_id_list)
    time_lb = '2021-11-01 00:00:00'
    time_ub = '2021-11-08 00:00:00'
    df = pd.DataFrame(columns = ['id', 'advertiser_id', 'ad_id', 'time', 'stat_cost','show_cnt', 'ctr', 'cpm_platform', 'click_cnt', 'pay_order_count', 'create_order_amount', 'prepay_and_pay_order_roi', 'prepay_order_count', 'prepay_order_amount', 'create_order_count', 'pay_order_amount', 'create_order_roi', 'dy_follow', 'convert_cnt', 'convert_cost', 'convert_rate', 'dy_share', 'dy_comment', 'dy_like', 'live_pay_order_cost_per_order', 'luban_live_enter_cnt', 'live_watch_one_minute_count', 'live_fans_club_join_cnt', 'luban_live_slidecart_click_cnt', 'luban_live_click_product_cnt', 'luban_live_comment_cnt', 'luban_live_share_cnt', 'luban_live_gift_cnt', 'luban_live_gift_amount', 'create_time', 'update_time'])
    for ad_id in ad_id_list:
        print("--------------- " + str(ad_id) + " (" + str(count) + " / " + str(length) + ") ---------------")
        sql = "SELECT * FROM qc_task_data_2111 WHERE ad_id = %s AND time >= %s AND time < %s"
        cur.execute(sql, (ad_id, time_lb, time_ub))
        row = cur.fetchall()
        if len(row) != 0:
            temp = pd.DataFrame(row, columns = ['id', 'advertiser_id', 'ad_id', 'time', 'stat_cost','show_cnt', 'ctr', 'cpm_platform', 'click_cnt', 'pay_order_count', 'create_order_amount', 'prepay_and_pay_order_roi', 'prepay_order_count', 'prepay_order_amount', 'create_order_count', 'pay_order_amount', 'create_order_roi', 'dy_follow', 'convert_cnt', 'convert_cost', 'convert_rate', 'dy_share', 'dy_comment', 'dy_like', 'live_pay_order_cost_per_order', 'luban_live_enter_cnt', 'live_watch_one_minute_count', 'live_fans_club_join_cnt', 'luban_live_slidecart_click_cnt', 'luban_live_click_product_cnt', 'luban_live_comment_cnt', 'luban_live_share_cnt', 'luban_live_gift_cnt', 'luban_live_gift_amount', 'create_time', 'update_time'])
            df = df.append(temp, ignore_index=True)
            print(df)
        count = count + 1

In [8]:
# 将qc_task_data_2111表中的数据存储到csv文件中
def Save_task_data_part_TJU(ad_id_list):
    fileLoc = "Data/task_data/卫仕-千川-自投.csv"
    f = open(fileLoc, 'w', encoding="utf-8", newline='')
    csv_writer = csv.writer(f)
    csv_writer.writerow(['id', 'advertiser_id', 'ad_id', 'time', 'stat_cost','show_cnt', 'ctr', 'cpm_platform', 'click_cnt', 'pay_order_count', 'create_order_amount', 'prepay_and_pay_order_roi', 'prepay_order_count', 'prepay_order_amount', 'create_order_count', 'pay_order_amount', 'create_order_roi', 'dy_follow', 'convert_cnt', 'convert_cost', 'convert_rate', 'dy_share', 'dy_comment', 'dy_like', 'live_pay_order_cost_per_order', 'luban_live_enter_cnt', 'live_watch_one_minute_count', 'live_fans_club_join_cnt', 'luban_live_slidecart_click_cnt', 'luban_live_click_product_cnt', 'luban_live_comment_cnt', 'luban_live_share_cnt', 'luban_live_gift_cnt', 'luban_live_gift_amount', 'create_time', 'update_time'])
    
    count = 1
    length = len(ad_id_list)
    time_lb = '2021-11-01 00:00:00'
    time_ub = '2021-11-08 00:00:00'
    for ad_id in ad_id_list:
        print("--------------- " + str(ad_id) + " (" + str(count) + " / " + str(length) + ") ---------------")
        sql = "SELECT * FROM qc_task_data_2111 WHERE ad_id = %s AND time >= %s AND time < %s"
        cur.execute(sql, (ad_id, time_lb, time_ub))
        row = cur.fetchall()
        if len(row) != 0:
            csv_writer.writerows(row)
#             print(row)
        count = count + 1
    f.close()
    print("File saving complete.")

In [9]:
# (1)部分含TJU计划
# 调用函数，获取部分含TJU计划团队的ad_id字段
advertiser_id = 1696827732541517
campaign_id_list = [1701275894994999, 1707671910546491, 1710617989499931, 1711602056635412, 1711602126695463, 1711602029279243, 1711602073722932, 1712237352138804, 1713511682447380, 1713511667950603, 1713429197611028, 1713600825933843, 1713600864418931, 1713875338932260, 1714058004496420, 1714320718938164,1714510799518772, 1715209794393219, 1714780228454423, 1715568631855127, 1715569577116676, 1715569580818493, 1716417266343975, 1716724754923555, 1716724754923555, 1717177943513099, 1717449957879828, 1717992978009150, 1717996231896083, 1717996196826132, 1719191133927444, 1719411371675709, 1719681886585915]
ad_id_list = Get_ad_id_part_TJU(advertiser_id, campaign_id_list)
print(ad_id_list)

# 调用函数，将计划效果数据存储到csv文件中
Save_task_data_part_TJU(ad_id_list)

# (2)(3)全部为/不含TJU计划
# 批量生成
# id_name = {'卫仕-千川-邦盟': 1696898004380686, '频度-卫仕小号': 1696993595431943, '卫仕nourse': 1697736606312455, '卫仕-千川-频度': 1702901918105607, '卫仕-千川-TJU': 1711685115933699, '卫仕-千川-自投一户（猫）': 1714195536577544, '卫仕-千川-自投一户（狗）': 1714296423936007, '卫仕-千川-谦行TJU': 1716820186570759}

# for i in id_name:
#     advertiser_id = id_name[i]
#     advertiser_name = list(id_name.keys())[list(id_name.values()).index(advertiser_id)]
#     print(advertiser_id, advertiser_name)
#     Save_task_data(advertiser_id, advertiser_name)

[1706421542930459, 1706421287801982, 1706421016406091, 1706420666718318, 1706418847390766, 1706418550368307, 1706418011364376, 1706417752134663, 1706414249639944, 1706413571346445, 1706409343513662, 1706408672446499, 1706407639081022, 1706405110404131, 1706404150047805, 1706361374578744, 1706360183426107, 1706358157828109, 1706356822550590, 1706353943392275, 1706352340193379, 1706351172682807, 1706340026615844, 1706338976600094, 1706337410904116, 1706333028867083, 1706316171809816, 1705552163298312, 1705551953902647, 1705548915383358, 1705548636867587, 1705546477597752, 1705545592762413, 1705543631138846, 1705518700352532, 1705498947554339, 1705498295496744, 1705496695957533, 1705490672454685, 1705430763983912, 1705430621328424, 1705430212237396, 1705429973655566, 1705368802178068, 1705368403566648, 1705367072577549, 1705365759835171, 1705323123576875, 1705323130125323, 1705323075262509, 1705323034751031, 1705323002098712, 1705322932353063, 1705321206755364, 1705321177107459, 170532111

--------------- 1706421287801982 (2 / 2304) ---------------
--------------- 1706421016406091 (3 / 2304) ---------------
--------------- 1706420666718318 (4 / 2304) ---------------
--------------- 1706418847390766 (5 / 2304) ---------------
--------------- 1706418550368307 (6 / 2304) ---------------
--------------- 1706418011364376 (7 / 2304) ---------------
--------------- 1706417752134663 (8 / 2304) ---------------
--------------- 1706414249639944 (9 / 2304) ---------------
--------------- 1706413571346445 (10 / 2304) ---------------
--------------- 1706409343513662 (11 / 2304) ---------------
--------------- 1706408672446499 (12 / 2304) ---------------
--------------- 1706407639081022 (13 / 2304) ---------------
--------------- 1706405110404131 (14 / 2304) ---------------
--------------- 1706404150047805 (15 / 2304) ---------------
--------------- 1706361374578744 (16 / 2304) ---------------
--------------- 1706360183426107 (17 / 2304) ---------------
--------------- 1706358157828109

--------------- 1709708511305767 (136 / 2304) ---------------
--------------- 1709772537285646 (137 / 2304) ---------------
--------------- 1709773542493235 (138 / 2304) ---------------
--------------- 1709779038585860 (139 / 2304) ---------------
--------------- 1709780057210891 (140 / 2304) ---------------
--------------- 1709780050868251 (141 / 2304) ---------------
--------------- 1709780046161943 (142 / 2304) ---------------
--------------- 1709780015194119 (143 / 2304) ---------------
--------------- 1709783007158327 (144 / 2304) ---------------
--------------- 1709782996570155 (145 / 2304) ---------------
--------------- 1709782989722664 (146 / 2304) ---------------
--------------- 1709782987712515 (147 / 2304) ---------------
--------------- 1709782980428819 (148 / 2304) ---------------
--------------- 1709782970010675 (149 / 2304) ---------------
--------------- 1709782957225011 (150 / 2304) ---------------
--------------- 1709782955958299 (151 / 2304) ---------------
--------

--------------- 1708692339760155 (269 / 2304) ---------------
--------------- 1708692702785572 (270 / 2304) ---------------
--------------- 1708700312088606 (271 / 2304) ---------------
--------------- 1708705179042827 (272 / 2304) ---------------
--------------- 1708751064110116 (273 / 2304) ---------------
--------------- 1708750943189028 (274 / 2304) ---------------
--------------- 1708751641749515 (275 / 2304) ---------------
--------------- 1708751373630487 (276 / 2304) ---------------
--------------- 1708751744199720 (277 / 2304) ---------------
--------------- 1708752919258155 (278 / 2304) ---------------
--------------- 1708753967773748 (279 / 2304) ---------------
--------------- 1708756058959939 (280 / 2304) ---------------
--------------- 1708757128613988 (281 / 2304) ---------------
--------------- 1708760232492087 (282 / 2304) ---------------
--------------- 1708774592041112 (283 / 2304) ---------------
--------------- 1708774947386395 (284 / 2304) ---------------
--------

--------------- 1709579214514222 (402 / 2304) ---------------
--------------- 1709584441213960 (403 / 2304) ---------------
--------------- 1709585807608891 (404 / 2304) ---------------
--------------- 1709592816836612 (405 / 2304) ---------------
--------------- 1709601696209976 (406 / 2304) ---------------
--------------- 1709603182318643 (407 / 2304) ---------------
--------------- 1709611505333283 (408 / 2304) ---------------
--------------- 1709608699956251 (409 / 2304) ---------------
--------------- 1709614739136627 (410 / 2304) ---------------
--------------- 1709614018596910 (411 / 2304) ---------------
--------------- 1709612239084603 (412 / 2304) ---------------
--------------- 1709610937080888 (413 / 2304) ---------------
--------------- 1709609724614667 (414 / 2304) ---------------
--------------- 1709657792335876 (415 / 2304) ---------------
--------------- 1709657712386099 (416 / 2304) ---------------
--------------- 1709657612276756 (417 / 2304) ---------------
--------

...

--------------- 1715321941107783 (1721 / 2304) ---------------
--------------- 1715321888329901 (1722 / 2304) ---------------
--------------- 1715363400479821 (1723 / 2304) ---------------
--------------- 1715363312938036 (1724 / 2304) ---------------
--------------- 1715367481469115 (1725 / 2304) ---------------
--------------- 1715367440829463 (1726 / 2304) ---------------
--------------- 1715367328351380 (1727 / 2304) ---------------
--------------- 1715367278779499 (1728 / 2304) ---------------
--------------- 1715367236562036 (1729 / 2304) ---------------
--------------- 1715464102679563 (1730 / 2304) ---------------
--------------- 1715464032295955 (1731 / 2304) ---------------
--------------- 1715463964230718 (1732 / 2304) ---------------
--------------- 1715464173893659 (1733 / 2304) ---------------
--------------- 1715464137167892 (1734 / 2304) ---------------
--------------- 1715503833127944 (1735 / 2304) ---------------
--------------- 1715507245838387 (1736 / 2304) --------

--------------- 1715636229057629 (1852 / 2304) ---------------
--------------- 1715636219626504 (1853 / 2304) ---------------
--------------- 1715636196807736 (1854 / 2304) ---------------
--------------- 1715636151679038 (1855 / 2304) ---------------
--------------- 1715816601637960 (1856 / 2304) ---------------
--------------- 1715816562433117 (1857 / 2304) ---------------
--------------- 1715816530324615 (1858 / 2304) ---------------
--------------- 1715816496984120 (1859 / 2304) ---------------
--------------- 1715816467232798 (1860 / 2304) ---------------
--------------- 1715913076787224 (1861 / 2304) ---------------
--------------- 1715913266222119 (1862 / 2304) ---------------
--------------- 1715913717142547 (1863 / 2304) ---------------
--------------- 1715913968259083 (1864 / 2304) ---------------
--------------- 1715913885194244 (1865 / 2304) ---------------
--------------- 1716000508975175 (1866 / 2304) ---------------
--------------- 1716002644508724 (1867 / 2304) --------

--------------- 1716839613190147 (1983 / 2304) ---------------
--------------- 1716839605205111 (1984 / 2304) ---------------
--------------- 1716839597569047 (1985 / 2304) ---------------
--------------- 1716839600576539 (1986 / 2304) ---------------
--------------- 1716839594340407 (1987 / 2304) ---------------
--------------- 1716839577552931 (1988 / 2304) ---------------
--------------- 1716839576002644 (1989 / 2304) ---------------
--------------- 1716839562074131 (1990 / 2304) ---------------
--------------- 1716839557885971 (1991 / 2304) ---------------
--------------- 1716839552652311 (1992 / 2304) ---------------
--------------- 1716839535501332 (1993 / 2304) ---------------
--------------- 1716839527924739 (1994 / 2304) ---------------
--------------- 1716839532047396 (1995 / 2304) ---------------
--------------- 1716839509774381 (1996 / 2304) ---------------
--------------- 1716839511415870 (1997 / 2304) ---------------
--------------- 1716839586804926 (1998 / 2304) --------

--------------- 1717632096488507 (2114 / 2304) ---------------
--------------- 1717632389486635 (2115 / 2304) ---------------
--------------- 1717632336495627 (2116 / 2304) ---------------
--------------- 1717632252796939 (2117 / 2304) ---------------
--------------- 1717632210676763 (2118 / 2304) ---------------
--------------- 1717721429878820 (2119 / 2304) ---------------
--------------- 1717721733228555 (2120 / 2304) ---------------
--------------- 1717721634734116 (2121 / 2304) ---------------
--------------- 1717721579537419 (2122 / 2304) ---------------
--------------- 1717721505255437 (2123 / 2304) ---------------
--------------- 1717721905387556 (2124 / 2304) ---------------
--------------- 1717812023702574 (2125 / 2304) ---------------
--------------- 1717812322681869 (2126 / 2304) ---------------
--------------- 1717812256920605 (2127 / 2304) ---------------
--------------- 1717812144331821 (2128 / 2304) ---------------
--------------- 1717812462089256 (2129 / 2304) --------

--------------- 1719411829899454 (2245 / 2304) ---------------
--------------- 1719411763247112 (2246 / 2304) ---------------
--------------- 1719411683984446 (2247 / 2304) ---------------
--------------- 1719411603187747 (2248 / 2304) ---------------
--------------- 1719448585135143 (2249 / 2304) ---------------
--------------- 1719448595217464 (2250 / 2304) ---------------
--------------- 1719448807560222 (2251 / 2304) ---------------
--------------- 1719448840710155 (2252 / 2304) ---------------
--------------- 1719448821971995 (2253 / 2304) ---------------
--------------- 1719448822939709 (2254 / 2304) ---------------
--------------- 1719448825064493 (2255 / 2304) ---------------
--------------- 1719448826940419 (2256 / 2304) ---------------
--------------- 1719448823476248 (2257 / 2304) ---------------
--------------- 1719448799780926 (2258 / 2304) ---------------
--------------- 1719448816314414 (2259 / 2304) ---------------
--------------- 1719448814151700 (2260 / 2304) --------